### Import libraries

In [1]:
import requests
import os
import time
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from tqdm.autonotebook import tqdm
from langchain.schema import Document as lcDocument
from langchain_community.utilities import GoogleSerperAPIWrapper
from bs4 import BeautifulSoup
from readability import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain

c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# Environment variables
os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pinecone_api_key
os.environ["SERPER_API_KEY"] = "caabb8481fd9568ce9e4f6149139098b7fd91f85"

### Download file GGUF

In [8]:
url = 'https://huggingface.co/mradermacher/Llama-3.1-MedPalm2-imitate-8B-Instruct-GGUF/resolve/main/Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf?download=true'
save_directory = 'models'
filename = 'Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

file_path = os.path.join(save_directory, filename)

if os.path.exists(file_path):
    print(f'Tệp {filename} đã tồn tại trong thư mục {save_directory}.')
else:
    response = requests.get(url, stream=True)

    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))  # Total size in bytes
        block_size = 8192  # Download in chunks of 8KB
        progress = 0  # Track the progress

        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    file.write(chunk)
                    progress += len(chunk)
                    percent_complete = (progress / total_size) * 100
                    print(f'Tải xuống: {percent_complete:.2f}%', end='\r')

        print(f'\nTệp đã được tải xuống và lưu tại {file_path}')
    else:
        print('Có lỗi xảy ra trong quá trình tải xuống.')


Tải xuống: 100.00%
Tệp đã được tải xuống và lưu tại models\Llama-3.1-MedPalm2-imitate-8B-Instruct.Q8_0.gguf


### Call model via API (llama_cpp server) with Langchain

In [3]:
llm = ChatOpenAI(base_url="http://zep.hcmute.fit/7500/v1", api_key="llama.cpp")

### Define the Offline_RAG Class

In [4]:
class HistoryChat:
    def __init__(self):
        self.history_chat  = []
        self.full_user_questions  = []
    
    def add_history(self, user_chat, bot_chat):
        self.history_chat.append(("user", user_chat))
        self.history_chat.append(("assistant", bot_chat))
        if len(self.history_chat) > 5:
            self.history_chat.pop(0)
        self.full_user_questions.append(user_chat)

    def get_user_question(self):
        return self.full_user_questions
    
    def get_chat_history(self):
        return self.history_chat

In [5]:
class WebSearchDB():
    def __init__(self):
        self.embedding = HuggingFaceEmbeddings()
        self.db = PineconeVectorStore(index_name='docs-rag-chatbot', embedding=self.embedding)
        self.text_slit = RecursiveCharacterTextSplitter(
                            separators = ["\n\n", "\n", " ", ".", ",", "\u200b", "\u3001", "\uff0e", "\u3002", ""],
                            chunk_size = 500,
                            chunk_overlap = 50,
                            length_function = len,
                            is_separator_regex = False)
        self.build_db()

    def build_db(self):
        namespace = "docs-link"
        try:
            results = self.db.similarity_search(query="test", namespace=namespace, k=1)
            if results:
                print(f"Namespace '{namespace}' đã tồn tại.")
                return self.db
            else:
                print(f"Namespace '{namespace}' tồn tại nhưng chưa có dữ liệu.")
                dummy_texts = ["Initial text for new namespace"]
                self.db.add_texts(dummy_texts, namespace=namespace)
                return self.db
        except Exception as e:
            print(f"Namespace '{namespace}' chưa tồn tại, sẽ tạo mới: {e}")
            dummy_texts = ["Initial text for new namespace"]
            self.db.add_texts(dummy_texts, namespace=namespace)
            return self.db
    
    def is_not_existed_url(self, url):
        results = self.db.similarity_search(query=url, namespace="docs-link", k=1)
        if results and results[0].page_content == url:
            return False
        return True
    
    def store_pinecone(self, docs, namespace):
        if docs:
            db = PineconeVectorStore.from_documents(documents=docs, embedding=self.embedding, index_name="docs-rag-chatbot", namespace=namespace)

    def store_web_content(self, web_links):
        documents = []
        if web_links:
            for link in web_links:
                content, title = self.read_content_from_web(link)
                if content:  # Check if content is valid
                    doc = lcDocument(page_content=content, metadata={"source": title})
                    chunks = self.text_slit.split_documents([doc]) 
                    cleaned_chunks = self.clean_data(chunks)
                
                    for i, chunk in enumerate(cleaned_chunks):
                        doc_chunk = lcDocument(
                            page_content=chunk.page_content,
                            metadata={"source": title, "chunk_id": i}
                        )
                        documents.append(doc_chunk)

            self.store_pinecone(documents, 'docs-store')

    def clean_data(self, chunks):
        cleaned_chunks = []
        for chunk in chunks:
            cleaned_content = chunk.page_content.replace("\n", " ")
            cleaned_content = re.sub(r'\s+', ' ', chunk.page_content)
            cleaned_content = cleaned_content.strip()
            cleaned_chunks.append(lcDocument(page_content=cleaned_content, metadata=chunk.metadata))
        return cleaned_chunks
        
    def read_content_from_web(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            doc = Document(response.content)
            html_content = doc.summary()
            title = doc.title()
            soup = BeautifulSoup(html_content, "html.parser")
            text_content = soup.get_text()
            return text_content, title
        return None, None

    def add_url(self, results):    
        links_document = []
        links = []
        for web in results['organic']:
            if self.is_not_existed_url(web['link']):
                link_doc = lcDocument(page_content=web['link'], metadata={'source': web['title']})
                links_document.append(link_doc)
                links.append(web['link'])
        self.store_pinecone(links_document, namespace='docs-link')
        self.store_web_content(links)

In [10]:
class Offline_RAG:
    def __init__(self, llm) -> None:
        self.llm = llm

        self.history_manager = HistoryChat()
        
        condense_question_template = """
        Với đoạn hội thoại sau và một câu hỏi tiếp theo, hãy diễn đạt lại câu hỏi tiếp theo để nó trở thành một câu hỏi độc lập.
        
        Lịch sử hội thoại:
        {chat_history}
        Câu hỏi tiếp theo: {question}
        Câu hỏi độc lập:"""
        
        self.condense_question_prompt = ChatPromptTemplate.from_template(condense_question_template)

        qa_template = """
        Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. 
        Dựa vào nội dung gợi ý trả lời bên dưới, hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi bên dưới.
        Nếu câu hỏi không liên quan đến sức khoẻ tâm lý sau sinh hoặc bạn không biết câu trả lời, hãy nói không biết hoặc từ chối trả lời.
        Nếu bạn cần thêm thông tin, hãy yêu cầu người dùng cung cấp thêm thông tin.
        Nếu người dùng sử dụng những từ ngữ không phù hợp, hãy yêu cầu họ sử dụng ngôn ngữ lịch sự.
        Câu trả lời của bạn cần phải có ít nhất 20 từ và không nên lãng tránh câu hỏi bằng cách khuyên người dùng đi bác sĩ.

        Lịch sử hội thoại:
        {chat_history}

        Nội dung gợi ý trả lời:
        {context}

        Câu hỏi: {question}
        """

        self.qa_prompt = ChatPromptTemplate.from_template(qa_template)

    def get_chain(self, retriever):
        rag_chain = ConversationalRetrievalChain.from_llm(
            llm,
            retriever,
            condense_question_prompt=self.condense_question_prompt,
            combine_docs_chain_kwargs={
                "prompt": self.qa_prompt,
            },
            return_source_documents=True,
        )
        return rag_chain
    
    def add_history(self, user_chat, bot_chat):
        self.history_manager.add_history(user_chat, bot_chat)
    
    def get_input_data(self,question):
        # search = GoogleSerperAPIWrapper()
        # search.k = 2
        # results = search.results(question)
        # WebSearchDB().add_url(results=results)
        chat_history = self.history_manager.get_chat_history()
        return {
            "chat_history": chat_history,
            "question": question
        }

### Define the VectorDB class using Pinecone

In [7]:
class VectorDB:
    def __init__(self,
                 embedding = HuggingFaceEmbeddings()) -> None:
        os.environ['PINECONE_API_KEY'] = "73bdec39-1f93-47fc-bd2f-f02883d7be83"
        self.pinecone_api_key = os.getenv('PINECONE_API_KEY')
        self.embedding = embedding
        self.db = self._build_db()
    
    def _build_db(self):
        db = PineconeVectorStore.from_existing_index(
            index_name="docs-rag-chatbot",
            namespace="docs-store",
            embedding=self.embedding
        )
        return db
    
    # tính similarity
    def get_retriever(self,
                      search_type: str = "similarity",
                      search_kwargs: dict = {"k": 5}
                      ):
        retriever =self.db.as_retriever(search_type=search_type,
                                        search_kwargs=search_kwargs)
        return retriever

c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Function build chain

In [8]:
def build_rag_chain(llm):
    retriever = VectorDB().get_retriever()
    off_rag = Offline_RAG(llm)
    rag_chain = off_rag.get_chain(retriever)
    return rag_chain, off_rag

### Run example

#### Not prompt and context

In [7]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory
)
sum_time = 0
n = 0
while True:
    question = input("You: ")

    if question == "exit":
        break

    start = time.time()
    response = conversation.invoke({
            "input": question
        })
    end = time.time()

    sum_time += end - start
    n += 1

    print(response)
    
    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")


{'input': 'Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?', 'history': '', 'response': 'Chào bạn! Tôi hiểu bạn đang lo lắng về những triệu chứng bạn đang trải qua sau khi sinh nở.  Đau đầu, mệt mỏi và suy nghĩ tiêu cực sau sinh là những triệu chứng rất phổ biến được gọi là **bệnh trầm cảm sau sinh** hoặc **baby blues**. \n\nTuy nhiên, tôi không phải là chuyên gia y tế nên tôi không thể chẩn đoán bệnh.  Điều quan trọng nhất là bạn cần đến gặp bác sĩ để được kiểm tra và tư vấn điều trị phù hợp. Bác sĩ có thể giúp bạn xác định nguyên nhân của những triệu chứng này và đưa ra lời khuyên tốt nhất cho trường hợp của bạn. \n\nHãy nhớ rằng, bạn không đơn độc và rất nhiều phụ nữ khác cũng trải qua những cảm xúc tương tự sau khi sinh con.  Đừng ngần ngại tìm kiếm sự trợ giúp từ người thân, bạn bè hoặc các chuyên gia y tế.\n\n\n'}
Time: 91.38202548027039
{'input': 'Hiện tại tôi không có thời gian đi bác sĩ, con tôi cứ khóc hoài, có cách nào để cho nó ngủ ngoan không ?', 'history': 'Hu

#### Not context

In [8]:
memory = ConversationBufferMemory()
qa_template = """
        Bạn là trợ lý AI hỗ trợ về sức khoẻ tâm lý sau sinh. 
        Hãy đưa ra câu trả lời và lời khuyên phù hợp cho câu hỏi của họ.
        Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.

        Lịch sử hội thoại:
        {history}

        Câu hỏi: {input}
        """
qa_prompt = ChatPromptTemplate.from_template(qa_template)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    prompt = qa_prompt
)

sum_time = 0
n = 0

while True:
    question = input("You: ")
    if question == "exit":
        break
    start = time.time()
    response = conversation.invoke({
            "input": question
        })
    end = time.time()
    sum_time += end - start
    n += 1
    print(response)
    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")

{'input': 'Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?', 'history': '', 'response': 'Tôi hiểu bạn đang cảm thấy rất khó khăn.  Đau đầu, mệt mỏi và suy nghĩ tiêu cực sau sinh là những triệu chứng thường gặp của **thoi ương trầm cảm** (postpartum depression).\n\nTuy nhiên, tôi không phải là chuyên gia y tế và không thể chẩn đoán bệnh. \n\n**Điều quan trọng nhất là bạn nên liên hệ với bác sĩ hoặc chuyên gia sức khỏe tâm thần để được tư vấn và hỗ trợ phù hợp.** Họ có thể giúp bạn xác định nguyên nhân của những triệu chứng bạn đang gặp phải và đưa ra phương pháp điều trị tốt nhất cho trường hợp của bạn.\n\nHãy nhớ rằng bạn không đơn độc và có rất nhiều nguồn lực sẵn sàng hỗ trợ bạn vượt qua giai đoạn này. \n'}
Time: 85.21116423606873
{'input': 'Hiện tại tôi không có thời gian đi bác sĩ, con tôi cứ khóc hoài, có cách nào để cho nó ngủ ngoan không ?', 'history': 'Human: Đau đầu, mệt mỏi, suy nghĩ tiêu cực sau sinh là bị bệnh gì?\nAI: Tôi hiểu bạn đang cảm thấy rất khó khăn.  Đ

#### Context

In [9]:
def update_datbase_with_websearch(user_questions, web_search_db):
    for question in user_questions:
        search = GoogleSerperAPIWrapper()
        search.k = 2
        results = search.results(question)
        web_search_db.add_url(results=results)

In [13]:
chain, off_rag = build_rag_chain(llm)
sum_time = 0
n = 0
web_search_db = WebSearchDB()
while True:
    question = input("You: ")
    if question == "exit":
        user_questions = off_rag.history_manager.get_user_question()
        update_datbase_with_websearch(user_questions, web_search_db)
        break
    
    start = time.time()
    response = chain.invoke(off_rag.get_input_data(question))
    end = time.time()

    sum_time += end - start
    n += 1

    print(response)
    
    off_rag.add_history(question, response["answer"])

    print(f"Time: {end - start}")

print(f"Average time: {sum_time/n}")

c:\Users\Admin\anaconda3\envs\chatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Namespace 'docs-link' đã tồn tại.
{'chat_history': [], 'question': 'Con tôi cứ quáy hoài ảnh hưởng đến tâm lý của tôi, bạn có thể chỉ cho tôi cách dỗ con ngủ ngoan không ', 'answer': 'Tôi hiểu rằng việc con thường xuyên quấy khóc ảnh hưởng rất nhiều đến tâm lý của bạn và đó là điều rất bình thường.  Mẹ sau sinh thường cảm thấy áp lực và mệt mỏi khi phải đối mặt với những tiếng khóc liên tục của con. \n\nĐể dỗ con ngủ ngoan, bạn có thể thử một số cách sau:\n\n* **Tạo môi trường ngủ yên tĩnh và thoải mái:** Giảm thiểu tiếng ồn, ánh sáng và nhiệt độ phù hợp cho bé.\n* **Thư giãn trước khi đi vào giấc ngủ:** Bạn có thể tạo thói quen tắm nước ấm, hát ru hoặc xoa bóp nhẹ nhàng cho con trước khi đưa vào giấc ngủ.\n* **Xây dựng chu trình giấc ngủ đều đặn:**  Cố gắng đưa con ngủ và dậy cùng giờ mỗi ngày, kể cả cuối tuần, để giúp bé lập trình được đồng hồ sinh học.\n* **Dùng biện pháp dỗ dành phù hợp:** Có nhiều kỹ thuật dỗ dành như vỗ về, bú bình, hát ru... Hãy thử nghiệm và tìm ra phương pháp 